<a href="https://colab.research.google.com/github/Megancodes2017/Megancodes2017/blob/main/Nola_Pred_SIERA_FIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install pybaseball

In [73]:
import pandas as pd
from pybaseball import pitching_stats, playerid_lookup
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [74]:
years = list(range(2018, 2025))
all_stats = pd.concat([pitching_stats(year) for year in years], ignore_index=True)
all_stats = all_stats[all_stats['IP'] > 100]

In [75]:
from pybaseball import pitching_stats

years = list(range(2015, 2025))
all_stats = pd.concat([pitching_stats(year) for year in years], ignore_index=True)
all_stats = all_stats[all_stats['G'] >= 10] # Filter for pitchers with at least 10 games played


all_stats['next_year'] = all_stats['Season'] + 1
future = all_stats[['Name', 'Season', 'ERA', 'FIP', 'SIERA']].rename(columns={'ERA': 'ERA_next_year', 'FIP': 'FIP_next_year', 'SIERA': 'SIERA_next_year', 'Season': 'next_year'})
merged = all_stats.merge(future, on=['Name', 'next_year'], how='left')

# Display the first few rows of the merged dataframe and its info to verify the merge and data types
display(merged.head())
display(merged.info())

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,...,botStf FS,botCmd FS,botOvr,botStf,botCmd,botxRV100,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO,next_year,ERA_next_year,FIP_next_year,SIERA_next_year
0,2036,2015,Clayton Kershaw,LAD,27,16,7,8.6,2.13,33,33,4,3,0,0,232.2,890,163,62,55,15,42,1,5,9,3,301,262,148,114,14,1089,2303,3392,115,16,18,4,11.64,1.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,NaN,NaN,NaN
1,4153,2015,Jake Arrieta,CHC,29,22,6,7.0,1.77,33,33,4,3,0,0,229.0,870,150,52,45,10,48,2,6,6,0,236,318,129,119,12,1202,2236,3438,118,22,14,3,9.28,1.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,3.10,3.52,3.94
2,3184,2015,David Price,- - -,29,18,5,6.7,2.45,32,32,3,1,0,0,220.1,888,190,70,60,17,47,2,3,4,0,225,243,219,139,24,1066,2322,3388,137,17,12,3,9.19,1.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,3.99,3.60,3.60
3,3137,2015,Max Scherzer,WSN,30,14,12,6.5,2.79,33,33,4,3,0,0,228.2,899,176,74,71,27,34,2,5,10,1,276,202,256,106,33,979,2380,3359,104,10,20,3,10.86,1.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,2.96,3.24,3.05
4,9434,2015,Dallas Keuchel,HOU,27,20,8,5.7,2.48,33,33,3,2,0,0,232.0,911,185,68,64,17,51,0,2,9,0,216,393,125,119,15,1289,2203,3492,150,23,5,1,8.38,1.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,4.55,3.87,3.77


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Columns: 397 entries, IDfg to SIERA_next_year
dtypes: float64(328), int64(65), object(4)
memory usage: 1.7+ MB


None

In [76]:
features = ['Season', 'Age', 'IP', 'ER', 'GB%', 'LD%', 'FB%', 'WHIP', 'WAR']
merged = merged.dropna(subset=features + ['SIERA_next_year'])

X = merged[features]
y = merged['SIERA_next_year']

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LGBMRegressor()
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000072 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 182, number of used features: 9
[LightGBM] [Info] Start training from score 3.970549
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

LGBMRegressor()

In [78]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {mse:.4f}')

Test MSE: 0.3185


In [79]:
# Predict for Aaron Nola
nola = all_stats[all_stats['Name'] == 'Aaron Nola'].sort_values(by='Season', ascending=False).head(1)

if not nola.empty:
  nola_input = nola[features]
  nola_SIERA_pred = model.predict(nola_input)
  print(f'Predicted SIERA for Aaron Nola: {nola_SIERA_pred[0]:.3f}')
else:
  print('No data available for Aaron Nola')

Predicted SIERA for Aaron Nola: 4.128


In [88]:
# Multi year targets - SIERA
for i in range(1, 4):
  all_stats[f'future_year_{i}'] = all_stats['Season'] + i

future_targets = []
for i in range(1, 4):
  future = all_stats[['Name', 'Season', 'SIERA']].rename(
      columns={'SIERA': f'SIERA_plus_{i}', 'Season': f'future_year_{i}'})
  future_targets.append(future)

merged = all_stats.copy()
for i in range(1, 4):
  merged = merged.merge(future_targets[i-1], on=['Name', f'future_year_{i}'], how='inner')


In [81]:
features = ['Season', 'Age', 'IP', 'ER', 'GB', 'LD', 'FB', 'SO', 'BB']
targets = ['SIERA_plus_1', 'SIERA_plus_2', 'SIERA_plus_3']

X = merged[features]

In [82]:
models = {}
predictions = {}

for i, target in enumerate(targets, start=1):
  y = merged[target]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = LGBMRegressor()
  model.fit(X_train, y_train)
  models[i] = model
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  print(f'Year +{i} Test MSE: {mse:.4f}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000055 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 125
[LightGBM] [Info] Number of data points in the train set: 45, number of used features: 9
[LightGBM] [Info] Start training from score 3.687778
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [83]:
nola = all_stats[all_stats['Name'] == 'Aaron Nola'].sort_values(by='Season', ascending=False).head(1)

if not nola.empty:
  nola_input = nola[features]
  for i in range(1, 4):
    nola_pred = models[i].predict(nola_input)
    print(f'Predicted SIERA for Aaron Nola in Year +{i}: {nola_pred[0]:.3f}')
else:
  print('Nola data not found')

Predicted SIERA for Aaron Nola in Year +1: 3.473
Predicted SIERA for Aaron Nola in Year +2: 3.461
Predicted SIERA for Aaron Nola in Year +3: 3.658


In [84]:
# Multi year targets - FIP
for i in range(1, 4):
  all_stats[f'future_year_{i}'] = all_stats['Season'] + i

future_targets = []
for i in range(1, 4):
  future = all_stats[['Name', 'Season', 'FIP']].rename(
      columns={'FIP': f'FIP_plus_{i}', 'Season': f'future_year_{i}'})
  future_targets.append(future)

merged = all_stats.copy()
for i in range(1, 4):
  merged = merged.merge(future_targets[i-1], on=['Name', f'future_year_{i}'], how='inner')

In [85]:
features = ['Season', 'Age', 'IP', 'HR', 'BB', 'HBP', 'SO']
targets = ['FIP_plus_1', 'FIP_plus_2', 'FIP_plus_3']

X = merged[features]

In [86]:
models = {}
predictions = {}

for i, target in enumerate(targets, start=1):
  y = merged[target]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = LGBMRegressor()
  model.fit(X_train, y_train)
  models[i] = model
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  print(f'Year +{i} Test MSE: {mse:.4f}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000033 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 45, number of used features: 7
[LightGBM] [Info] Start training from score 3.473556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [87]:
nola = all_stats[all_stats['Name'] == 'Aaron Nola'].sort_values(by='Season', ascending=False).head(1)

if not nola.empty:
  nola_input = nola[features]
  for i in range(1, 4):
    nola_pred = models[i].predict(nola_input)
    print(f'Predicted FIP for Aaron Nola in Year +{i}: {nola_pred[0]:.3f}')
else:
  print('Nola data not found')

Predicted FIP for Aaron Nola in Year +1: 3.052
Predicted FIP for Aaron Nola in Year +2: 2.978
Predicted FIP for Aaron Nola in Year +3: 3.438
